# Sector Visualization

In this example, we will parse the database and visualize each sector's averaage times.

## Step 0: Global variables

In [ ]:
database_path = 'bike_data.db'

## Step 1: Import required libraries and the database

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from ipyleaflet import Map, Marker, Polygon
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math

# SQLAlchemy connectable 
conn = create_engine( 'sqlite:///' + database_path ).connect()

## Step 2: Import Sector Data

In [3]:
sector_list = pd.read_sql_table( 'sector_list',conn)
centroid = [0, 0]
num_points = 0

polygons = []

counter = 0.0
for sector in sector_list['sector_id']:
    
    sector_points = pd.read_sql_table( sector, conn )
    
    poly_draw = Polygon( )
    centroid = [centroid[0] + sector_points['latitude'].mean()  * len(sector_points),
                centroid[1] + sector_points['longitude'].mean() * len(sector_points)]
    num_points += len( sector_points )
    
    color_code = matplotlib.colors.rgb2hex(plt.get_cmap('hsv')( counter / len(sector_list) ))
    poly_points = [ ( row['latitude'], row['longitude'] ) for index, row in sector_points.iterrows() ]
    polygons.append( Polygon( locations=poly_points, 
                              color=color_code,
                              fill_color=color_code ) )
    counter += 1.0

Here is a handy visualization of the designated sectors.

In [4]:
centroid_pt = (centroid[0]/num_points, 
               centroid[1]/num_points)

#  Build Map Visualization    
m = Map( center=centroid_pt, zoom=11)
for polygon in polygons:
    m.add_layer(polygon);
    
m.layout.height="550px"    
m 

Map(center=[39.68022141177373, -104.91524623127385], controls=(ZoomControl(options=['position', 'zoom_in_text'…

For each sector, we need to construct tracks

In [5]:
def Build_Segment( conn, point_list, dataset_id, sector_id ):
    
    #  Get a list of all points with that sector-id
    sql_query = 'SELECT * FROM point_list WHERE (sector_id = \'{}\') and (dataset_id = {})'.format( sector_id, dataset_id )
    points_in_sector = pd.read_sql_query( sql_query, conn )
    print(points_in_sector)
    
    return points_in_sector

Construct the tracks

In [6]:
#  For each segment, we need to create a track for each dataset
dataset_ids = pd.read_sql_query('SELECT DISTINCT dataset_id FROM point_list', conn)

#  Load the entire list of points into one dataframe for quicker access
point_list = pd.read_sql_table( 'point_list', conn )

for sector_id in sector_list['sector_id']:
    
    #  For each dataset
    for dataset_id in dataset_ids['dataset_id']:
        
        #  Create a full track of the segment
        sql_query = 'SELECT * FROM point_list WHERE (sector_id = \'{}\') and (dataset_id = {}) ORDER BY timestamp'.format( sector_id, dataset_id )
        points_in_sector = pd.read_sql_query( sql_query, conn )
        
        #  from the point list, we can compute time spent in sector
        points_in_sector['timestamp'] =  pd.to_datetime(points_in_sector['timestamp'])
        time_in_sector = points_in_sector['timestamp'].max() - points_in_sector['timestamp'].min()
        
        #  We can build a linked-list of points
        distance_sector = 0
        p0 = None
        p1 = None
        counter = 0
        for point in points_in_sector.itertuples( index=False ):
            #print('Point: {}, Lon: {}, Lat: {}'.format(point, point[0], point[1]))
            if p0 is None:
                p0 = point
            else:
                p1 = point
                try:
                    distance_sector += Distance_Vincenty( ( p0[2], p0[1] ), 
                                                          ( p1[2], p1[1] ) )
                except Exception as e:
                    print( 'Exception: {}'.format(e))
                    print( 'Failed to parse. Counter: {}, Index P0: {}, Index P1: {}'.format( counter, p0[0], p1[0]) )
                p0 = point
            counter += 1
            
        print('Sector: {}, Dataset: {}, Distance: {}'.format( sector_id, dataset_id, distance_sector ) )
        #  We can also find the distance run through sector
        #distance_in_sector
        

Sector: sector_0, Dataset: 0.0, Distance: 3649.276116781399
Sector: sector_0, Dataset: 1.0, Distance: 0
Sector: sector_1, Dataset: 0.0, Distance: 3347.4011820373726
Sector: sector_1, Dataset: 1.0, Distance: 2948.195191112837
Sector: sector_2, Dataset: 0.0, Distance: 2948.016750182945
Sector: sector_2, Dataset: 1.0, Distance: 3031.246009229241
Sector: sector_3, Dataset: 0.0, Distance: 3980.784376813766
Sector: sector_3, Dataset: 1.0, Distance: 3877.8274864821688
Sector: sector_4, Dataset: 0.0, Distance: 3584.5491472099307
Sector: sector_4, Dataset: 1.0, Distance: 3571.634784463647
Sector: sector_5, Dataset: 0.0, Distance: 4117.7276027881135
Sector: sector_5, Dataset: 1.0, Distance: 4072.9681893778697
Sector: sector_6, Dataset: 0.0, Distance: 2134.9123142436347
Sector: sector_6, Dataset: 1.0, Distance: 2080.6547772961853
Sector: sector_7, Dataset: 0.0, Distance: 2936.0076124127695
Sector: sector_7, Dataset: 1.0, Distance: 2807.167202219776
Sector: sector_8, Dataset: 0.0, Distance: 2294.9

## References

* Vincenty Formula Code: https://nathanrooy.github.io/posts/2016-12-18/vincenty-formula-with-python/